In [3]:
#Setting up OpenAI API Key
from dotenv import load_dotenv

load_dotenv('env.sh')  # Specify path to your env file

True

In [4]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
            model_name="gpt-3.5-turbo",
            temperature=0.0,
            verbose=True
        )

# Prompts

## PromptTemplate

Prompt templates are predefined recipes for generating prompts for language models.

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.")

prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [6]:
print(model.invoke(prompt_template.format(adjective="funny", content="chickens")).content)

Why did the chicken join a band?

Because it had the drumsticks!


In [7]:
print(model.invoke(prompt_template.format(adjective="sad", content="chickens")).content)

Why did the chicken cross the road? To get to the other side, where it found out its eggs had been stolen.


## Roles in PromptTemplate

In [8]:
from langchain_core.prompts import (HumanMessagePromptTemplate,
SystemMessagePromptTemplate,
AIMessagePromptTemplate,
ChatPromptTemplate)

In [9]:
system_template = SystemMessagePromptTemplate(
                prompt=PromptTemplate.from_template("You are an expert translator. Your task is to translate sentences to Hindi. Never translate any abusive word."))

human_template = HumanMessagePromptTemplate(
                prompt=PromptTemplate.from_template("{input}"))

In [10]:
chat_template = ChatPromptTemplate.from_messages(
    [system_template, human_template])

In [11]:
print(model.invoke(chat_template.format_messages(input="How are you doing?")).content)

आप कैसे हैं?


In [12]:
prompt = """
System: You are an expert translator. Your task is to translate sentences to Hindi. Never translate any abusive word.
Human:How are you doing?
AI:
"""

In [13]:
print(model.invoke(prompt).content)

कैसे हो आप?


# Chains

In [14]:
from langchain import LLMChain

In [15]:
chain = LLMChain(
    llm=model,
    prompt=chat_template,
    verbose=True,
)

In [16]:
response = chain.invoke({"input":"Weather is nice today."})
print("Output:-", response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an expert translator. Your task is to translate sentences to Hindi. Never translate any abusive word.
Human: Weather is nice today.

> Finished chain.
Output:- आज मौसम अच्छा है।


# Output Parsers

In [17]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [24]:
output_parser = CommaSeparatedListOutputParser()

In [25]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [27]:
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

In [28]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=output_parser,
    verbose=True,
)

In [29]:
response = chain.invoke({"subject": "Programming Languages"})
print("Output:-", response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
List five Programming Languages.
Your response should be a list of comma separated values, eg: `foo, bar, baz`

> Finished chain.
Output:- ['Java', 'Python', 'C++', 'JavaScript', 'Ruby']


In [31]:
k = model.invoke("List five Programming Languages. Your response should be a list of comma separated values, eg: `foo, bar, baz`").content
k.split(",")

['Java', ' Python', ' C++', ' JavaScript', ' Ruby']

# Memory

In [32]:
from langchain.memory import ConversationBufferMemory

In [33]:
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

In [34]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation = LLMChain(
    llm=model,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [35]:
response = conversation({"question": "Hi"})
print("Output:-", response['text'])

/usr/local/miniconda/envs/python3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: Hi

> Finished chain.
Output:- Hello! How are you today?


In [36]:
response = conversation({"question": "I am good"})
print("Output:-", response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: Hi
AI: Hello! How are you today?
Human: I am good

> Finished chain.
Output:- That's great to hear! What's on your mind today?


In [37]:
response = conversation({"question": "Not much. Boring!"})
print("Output:-", response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: Hi
AI: Hello! How are you today?
Human: I am good
AI: That's great to hear! What's on your mind today?
Human: Not much. Boring!

> Finished chain.
Output:- I understand, we all have those days. Is there anything I can do to help make your day a little less boring?


In [38]:
response = conversation({"question": "What was my first message?"})
print("Output:-", response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: Hi
AI: Hello! How are you today?
Human: I am good
AI: That's great to hear! What's on your mind today?
Human: Not much. Boring!
AI: I understand, we all have those days. Is there anything I can do to help make your day a little less boring?
Human: What was my first message?

> Finished chain.
Output:- Your first message was "Hi". How can I assist you further?


In [40]:
output_parser.parse("1,2")

['1,2']